# Napari image viewer

Uses the octopuslite dask image loader to automatically sort and load images from multichannel time lapse experiments. Can also use btrack to load associated tracks and segmentation images.

In [1]:
import napari
from octopuslite import DaskOctopusLiteLoader
import glob, os
import numpy as np
from datetime import datetime
from skimage.io import imread
from tqdm.auto import tqdm
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

def compute(dask_image_stack, N = None):
    
    """lazy hack to get around dask not loading transformed stacks in one go"""
    
    if N == None:
        N = (0, len(dask_image_stack) )
        
    return np.stack([frame.compute().astype(np.uint8) for frame in tqdm(dask_image_stack[N[0]:N[1]], total = N[1]-N[0])], axis = 0)

In [2]:
### load experiment information
expt_info = pd.read_csv('extrusion_expts.csv', )
del expt_info['Unnamed: 0']
expt_info

,EXP n˚,POSITION,CELL TYPE
0,ND0013,Pos3,90:10 wt:ras+
1,ND0013,Pos4,90:10 wt:ras+
2,ND0013,Pos5,90:10 wt:ras+
3,ND0013,Pos6,90:10 wt:ras+
4,ND0013,Pos7,90:10 wt:ras+
...,...,...,...
101,ND0025,Pos9,97.5:2.5 wt:ras+
102,ND0025,Pos10,97.5:2.5 wt:ras+
103,ND0025,Pos11,97.5:2.5 wt:ras+
104,ND0025,Pos12,97.5:2.5 wt:ras+


In [7]:
### Set root directory (server or local)
root_dir = '/home/nathan/data/kraken/ras/'#'/media/nathan/Transcend/Nathan/data/kraken/ras'
### make output directory for .csv list of extrusion coordinates
os.makedirs('extrusions', exist_ok = True)

In [4]:
### exclude certain expts if you wish
expt_info = expt_info[((expt_info['CELL TYPE'] == '99:1 wt:ras+'))]# & (expt_info['POSITION'] == 'Pos3')& (expt_info['POSITION'] == 'Pos4'))]

# save out cell ID to dict

In [5]:
import btrack
import json
from btrack.utils import tracks_to_napari
extr_dict_address = '/home/nathan/data/kraken/ras/experiment_information/extrusions/extrusion_dict.json'

In [6]:
## backup
with open(extr_dict_address) as f:
    extru_dict = json.load(f)
extru_dict

{'ND0025_Pos6_620_RFP': '874',
 'ND0025_Pos6_262_RFP': '968',
 'ND0025_Pos6_2_RFP': '940',
 'ND0025_Pos7_930_RFP': '1030',
 'ND0025_Pos7_958_RFP': '1136',
 'ND0025_Pos5_1_RFP': '420',
 'ND0014_Pos5_11_RFP': '194.0',
 'ND0014_Pos5_488_RFP_frag': '276.0',
 'ND0014_Pos5_1317_RFP_frag': '592.0',
 'ND0014_Pos5_2535_RFP_frag': '898.0',
 'ND0014_Pos7_1_RFP_frag': '145.0',
 'ND0014_Pos8_482_RFP': '900.0',
 'ND0014_Pos8_481_RFP': '1426.0',
 'ND0014_Pos9_1_RFP': '877.0',
 'ND0017_Pos7_1012_RFP_frag': '679.0',
 'ND0017_Pos7_547_RFP': '615.0',
 'ND0017_Pos7_814_RFP': '775.0',
 'ND0017_Pos7_1627_RFP_frag': '775.0',
 'ND0017_Pos7_1627_RFP': '952.0',
 'ND0021_Pos4_4_RFP': '860.0',
 'ND0021_Pos7_321_RFP': '1151.0',
 'ND0021_Pos7_320_RFP': '1139.0',
 'ND0021_Pos7_177_RFP': '1181.0',
 'ND0021_Pos7_178_RFP': '1146.0',
 'ND0021_Pos7_507_RFP': '1234.0',
 'ND0021_Pos7_506_RFP': '1234.0',
 'ND0021_Pos7_598_RFP': '1293.0',
 'ND0021_Pos7_599_RFP': '1293.0',
 'ND0022_Pos3_4_RFP_frag': '31.0',
 'ND0022_Pos3_907_

In [7]:
# with open(extr_dict_address, 'w',) as json_file:
#         json.dump(extru_dict, json_file)

In [11]:
### iterate over each competitive experiment
for i, expt_pos in tqdm(expt_info.iterrows(), desc = 'Progress throughout valid expts', total = len(expt_info)):
    ### get expt info
    expt = expt_pos['EXP n˚']
    pos = expt_pos['POSITION']
    expt, pos = 'ND0025', 'Pos6'
    print(f'Loading experiment {expt} {pos}')
    ### load images using dask
    image_path = f'{root_dir}/{expt}/{pos}/{pos}_images'
    transform_path = f'{root_dir}/{expt}/{pos}/transform_tensor.npy'
    tracks_path = f'{root_dir}/{expt}/{pos}/tracks.h5'
#     if not os.path.exists(transform_path) or not os.path.exists(tracks_path):
#         print('No transformation tensor exists, skipping experiment')
#         continue
#     if len([i for i in extru_dict if f'{expt}_{pos}' in i]) > 0:
#         print('Looks like this expt already done, skipping to next')
#         continue
    images = DaskOctopusLiteLoader(image_path, transforms=transform_path, crop = (1200, 1600))
    ### (optional) load images into memory for quicker scrolling
    stack_gfp = images['gfp']#compute(images['gfp'])
    stack_rfp = images['rfp']#compute(images['rfp'])
    with btrack.dataio.HDF5FileHandler(tracks_path,#.replace('tracks', 'tracks_ras_improved'), 
                                   'r', obj_type='obj_type_2'
                                  ) as h:
        tracks = h.tracks
    visualise_rfp_tracks, rfp_properties, rfp_graph = tracks_to_napari(tracks, ndim = 2)
    ### open napari
    viewer = napari.Viewer()
    ### add images to napari
    gfp_layer = viewer.add_image(stack_gfp, name=f"gfp {expt}, {pos}", 
                     contrast_limits=[0,119],
                     blending = 'additive', colormap = 'green')
    mask_layer = viewer.add_labels((images['mask']==2).astype(int), name = 'masks',
               blending = 'additive',)
    viewer.add_image(stack_rfp, name=f"rfp {expt}, {pos}", 
                     contrast_limits=[0,55], 
                     blending = 'additive', colormap = 'magenta')
    tracker_layer = viewer.add_tracks(visualise_rfp_tracks, properties=rfp_properties,)# graph = rfp_graph)
    
    @viewer.bind_key('r')
    def write_apoptosis(event=None):
        position = viewer.cursor.position
        cell_ID = tracker_layer._get_value(position)
        time = position[0]
        cell_track = [track for track in tracks if track.ID == cell_ID][0]
        if cell_track.t[0] > time - 150:
            import warnings
            warnings.warn(f"This track only extends back {time - cell_track.t[0]} frames/ {(time - cell_track.t[0])/15} hours")
            print(f"This track only extends back {time - cell_track.t[0]} frames/ {(time - cell_track.t[0])/15} hours")
            extru_dict[f'{expt}_{pos}_{cell_ID}_RFP_frag'] = str(time)
            print(f'{expt}_{pos}_{cell_ID}_RFP_frag:{int(time)}')
        else:
            extru_dict[f'{expt}_{pos}_{cell_ID}_RFP'] = str(time)
            print(f'{expt}_{pos}_{cell_ID}_RFP:{int(time)}')
    ### open at start of time lapse
    viewer.dims.current_step = (0,0,0)
    ### only open next experiment after closing current one
    viewer.show(block=True)
    ### 
    with open(extr_dict_address, 'w',) as json_file:
        json.dump(extru_dict, json_file)
        
    break

Progress throughout valid expts:   0%|          | 0/51 [00:00<?, ?it/s]

Loading experiment ND0025 Pos6
Using cropping: (1200, 1600)


[INFO][2022/05/25 09:35:09 am] Opening HDF file: /home/nathan/data/kraken/ras//ND0025/Pos6/tracks.h5...
[INFO][2022/05/25 09:35:09 am] Loading tracks/obj_type_2
[INFO][2022/05/25 09:35:09 am] Loading objects/obj_type_2 (5950, 5) (5950 filtered: None)
[INFO][2022/05/25 09:35:09 am] Closing HDF file: /home/nathan/data/kraken/ras//ND0025/Pos6/tracks.h5


This track only extends back 28.0 frames/ 1.8666666666666667 hours
ND0025_Pos6_696_RFP_frag:874
ND0025_Pos6_2_RFP:940
ND0025_Pos6_634_RFP:1010


In [17]:
len(extru_dict)

92

# save out points layer

In [ ]:
### iterate over each competitive experiment
for i, expt_pos in tqdm(expt_info.iterrows(), desc = 'Progress throughout valid expts', total = len(expt_info)):
    ### get expt info
    expt = expt_pos['EXP n˚']
    pos = expt_pos['POSITION']
    print(f'Loading experiment {expt} {pos}')
    ### load images using dask
    image_path = f'{root_dir}/{expt}/{pos}/{pos}_images'
    ### check if already done this experiment
    fn = f'extrusions/{expt}_{pos}_extrusions.csv'
    if os.path.exists(fn):
        date_time = datetime.now().strftime("%m.%d.%Y.%H:%M:%S")
        fn = f'extrusions/{expt}_{pos}_extrusions_updated_{date_time}.csv'
        continue ### comment out this line if you dont want to skip if already exists
    transform_path = f'{root_dir}/{expt}/{pos}/transform_tensor.npy'
    if not os.path.exists(transform_path):
        print('No transformation tensor exists, skipping experiment')
        continue
    images = DaskOctopusLiteLoader(image_path, transforms=transform_path)
    ### (optional) load images into memory for quicker scrolling
    stack_gfp = images['gfp']#compute(images['gfp'])
    stack_rfp = images['rfp']#compute(images['rfp'])
    ### open napari
    viewer = napari.Viewer()
    ### add images to napari
    viewer.add_image(stack_gfp, name=f"gfp {expt}, {pos}", 
                     contrast_limits=[0,119],
                     blending = 'additive', colormap = 'green')
#     viewer.add_image(stack_bf, name=f"bf {expt}, {pos}", 
#                      contrast_limits=[0,255],
#                      blending = 'additive')
    viewer.add_image(stack_rfp, name=f"rfp {expt}, {pos}", 
                     contrast_limits=[0,55], 
                     blending = 'additive', colormap = 'magenta')
    ### add point layer for labelling extrusion
    points_layer = viewer.add_points(np.empty((0,3)), ndim = 3,face_color='orange', name = 'extrusions', out_of_slice_display=True)
    ### open at start of time lapse
    viewer.dims.current_step = (0,0,0)
    ### only open next experiment after closing current one
    viewer.show(block=True)
    ### save out list of extrusion coords
    points_layer.save(fn)